# Délka dožití

Připravím si [data](https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who) a pak použiji náhodný les, lineární regresi, KNN a rozhodovací strom k predikci délky dožití v různých zemích a letech.

Dataset má následující příznaky:

- Country - země
- Year - rok
- Status - rozvinutá/rozvojová země
- **Life expectancy** - délka dožití v letech
- Adult Mortality - úmrtnost dospělých, obě pohlaví (počet úmrtí mezi 15 a 60 lety věku na 1000 osob)
- infant deaths - úmrtnost kojenců (počet úmrtí na 1000 osob)
- Alcohol - spotřeba alkoholu jednoho člověka v litrech
- percentage expenditure - výdaje na zdravotnictví (procento HDP na obyvatele)
- Hepatitis B - podíl jednoletých dětí očkovaných na hepatitis B v procentech
- Measles - počet nahlášených případů spalniček na 1000 obyvatel
- BMI - průměrné BMI celé populace
- under-five deaths - počet smrtí dětí mladších pěti let na 1000 obyvatel
- Polio - podíl jednoletých dětí očkovaných na dětskou obrnu v procentech
- Total expenditure - podíl výdajů vlády na zdravotnictví (procento z celkových vládních výdajů)
- Diphtheria - podíl jednoletých dětí očkovaných na záškrt, tetanus a černý kašel v procentech
- HIV/AIDS - počet smrtí na HIV/AIDS na 1000 živě narozených dětí (0-4 roky)
- GDP - HDP na obyvatele v USD
- Population - počet obyvatel země
- thinness 1-19 years - podíl obyvatel ve věku 10-19 let trpících podváhou v procentech
- thinness 5-9 years - podíl dětí ve věku 5-9 let trpících podváhou v procentech
- Income composition of resources - Human Development Index z hlediska příjmového složení zdrojů (0 až 1)
- Schooling - počet let školní docházky

Dvojice *Country* a *Year* bude sloužit jako index.

In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn import metrics, datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tqdm import tqdm
%matplotlib inline

# Příprava dat

Data načtu, rozdělím a zpracuji. Hodnoty musím převést na číselnou reprezentaci.

In [2]:
data = pd.read_csv("data.csv")
display(data.head())
data.info()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2718 entries, 0 to 2717
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          2718 non-null   object 
 1   Year                             2718 non-null   int64  
 2   Status                           2718 non-null   object 
 3   Life expectancy                  2718 non-null   float64
 4   Adult Mortality                  2718 non-null   float64
 5   infant deaths                    2718 non-null   int64  
 6   Alcohol                          2558 non-null   float64
 7   percentage expenditure           2718 non-null   float64
 8   Hepatitis B                      2187 non-null   float64
 9   Measles                          2718 non-null   int64  
 10  BMI                              2686 non-null   float64
 11  under-five deaths                2718 non-null   int64  
 12  Polio               

## Rozdělení dat

Data si rozdělím na trénovací, validační a testovací množinu v poměru 6:2:2. *Life expectancy* je vysvětlovaná proměnná, rovnou ji oddělím. Je typu `float64`. Odteď budu nahlížet jen do trénovací množiny. Vidím, že všechny příznaky kromě *Country* a *Status* jsou číselné. Zároveň jsou v mnoha sloupcích chybějící hodnoty.

In [3]:
Xdata = data.drop("Life expectancy", axis=1)
Ydata = data["Life expectancy"]

random_seed = 333

Xtrain, Xrest, Ytrain, Yrest = train_test_split(Xdata, Ydata, test_size=0.4, random_state=random_seed)
Xval, Xtest, Yval, Ytest = train_test_split(Xrest, Yrest, test_size=0.5, random_state=random_seed)

display(Xtrain.head())
display(Xtrain.info())
display(Xtrain.describe())
display(Xtrain.nunique())

,Country,Year,Status,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
1288,Kuwait,2011,Developing,85.0,1,0.02,2785.097712,99.0,32,69.0,...,99.0,2.62,99.0,0.1,48268.591200,NaN,3.3,3.2,0.792,13.4
1088,India,2015,Developing,181.0,910,NaN,0.000000,87.0,90387,18.7,...,86.0,NaN,87.0,0.2,1613.188780,1395398.0,26.7,27.3,0.615,11.6
1018,Guinea-Bissau,2001,Developing,32.0,5,2.55,0.868708,NaN,126,17.9,...,56.0,5.40,53.0,3.6,39.486749,1267512.0,1.0,1.0,0.000,6.7
341,Brunei Darussalam,2012,Developing,79.0,0,0.01,3192.634413,99.0,1,38.2,...,99.0,2.30,99.0,0.1,47651.259900,NaN,5.8,5.3,0.852,14.4
1253,Kenya,2014,Developing,255.0,56,0.01,170.962662,92.0,354,21.3,...,93.0,5.72,92.0,2.9,1335.645800,462425.0,7.8,7.7,0.546,11.1


<class 'pandas.core.frame.DataFrame'>
Index: 1630 entries, 1288 to 1804
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          1630 non-null   object 
 1   Year                             1630 non-null   int64  
 2   Status                           1630 non-null   object 
 3   Adult Mortality                  1630 non-null   float64
 4   infant deaths                    1630 non-null   int64  
 5   Alcohol                          1530 non-null   float64
 6   percentage expenditure           1630 non-null   float64
 7   Hepatitis B                      1313 non-null   float64
 8   Measles                          1630 non-null   int64  
 9   BMI                              1612 non-null   float64
 10  under-five deaths                1630 non-null   int64  
 11  Polio                            1618 non-null   float64
 12  Total expenditure     

None

,Year,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
count,1630.000000,1630.000000,1630.000000,1530.000000,1630.000000,1313.000000,1630.000000,1612.000000,1630.000000,1618.000000,1512.000000,1618.000000,1630.000000,1373.000000,1.241000e+03,1612.000000,1612.000000,1539.000000,1539.000000
mean,2007.133742,165.491411,32.076687,4.462379,702.786073,80.396040,2610.640491,37.614516,44.660736,82.370210,5.861349,81.765142,1.813436,7217.423234,1.360811e+07,5.030583,5.090323,0.616615,11.859844
std,4.539496,126.439351,116.431395,4.000243,1931.858228,26.152666,12566.093020,19.818000,158.658556,23.252762,2.413356,24.328747,5.194869,14187.458555,7.109659e+07,4.535993,4.660782,0.216347,3.326019
min,2000.000000,1.000000,0.000000,0.010000,0.000000,2.000000,0.000000,1.800000,0.000000,3.000000,0.370000,2.000000,0.100000,4.613575,3.400000e+01,0.100000,0.100000,0.000000,0.000000
25%,2003.000000,72.000000,0.000000,0.822500,5.349201,76.000000,0.000000,18.700000,0.000000,77.000000,4.230000,77.000000,0.100000,456.512487,2.337170e+05,1.700000,1.700000,0.487000,10.100000
50%,2007.000000,144.000000,3.000000,3.510000,67.717498,92.000000,21.500000,42.150000,4.000000,93.000000,5.670000,93.000000,0.100000,1643.758397,1.431628e+06,3.500000,3.400000,0.669000,12.200000
75%,2011.000000,226.750000,24.750000,7.337500,438.115433,97.000000,406.750000,55.500000,33.000000,97.000000,7.322500,97.000000,0.800000,5649.978486,7.775327e+06,7.400000,7.400000,0.768500,14.000000
max,2015.000000,723.000000,1700.000000,16.990000,19099.045060,99.000000,212183.000000,77.600000,2200.000000,99.000000,17.000000,99.000000,50.600000,119172.741800,1.293859e+09,27.300000,28.300000,0.938000,20.700000


Country                             183
Year                                 16
Status                                2
Adult Mortality                     376
infant deaths                       168
Alcohol                             799
percentage expenditure             1294
Hepatitis B                          83
Measles                             619
BMI                                 542
under-five deaths                   205
Polio                                72
Total expenditure                   668
Diphtheria                           79
HIV/AIDS                            155
GDP                                1373
Population                         1238
thinness  1-19 years                181
thinness 5-9 years                  187
Income composition of resources     540
Schooling                           163
dtype: int64

## Index

Předpovídám délku dožití pro zemi a rok. Dvojice příznaků *Country* a *Year* mi tedy bude sloužit jako index.

In [4]:
Xtrain = Xtrain.set_index(["Country", "Year"], verify_integrity=True)
Xval = Xval.set_index(["Country", "Year"], verify_integrity=True)
Xtest = Xtest.set_index(["Country", "Year"], verify_integrity=True)

Xtrain.head()

,,Status,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
Country,Year,,,,,,,,,,,,,,,,,,,
Kuwait,2011,Developing,85.0,1,0.02,2785.097712,99.0,32,69.0,1,99.0,2.62,99.0,0.1,48268.591200,NaN,3.3,3.2,0.792,13.4
India,2015,Developing,181.0,910,NaN,0.000000,87.0,90387,18.7,1100,86.0,NaN,87.0,0.2,1613.188780,1395398.0,26.7,27.3,0.615,11.6
Guinea-Bissau,2001,Developing,32.0,5,2.55,0.868708,NaN,126,17.9,9,56.0,5.40,53.0,3.6,39.486749,1267512.0,1.0,1.0,0.000,6.7
Brunei Darussalam,2012,Developing,79.0,0,0.01,3192.634413,99.0,1,38.2,0,99.0,2.30,99.0,0.1,47651.259900,NaN,5.8,5.3,0.852,14.4
Kenya,2014,Developing,255.0,56,0.01,170.962662,92.0,354,21.3,79,93.0,5.72,92.0,2.9,1335.645800,462425.0,7.8,7.7,0.546,11.1


## Příznak *Status*

Příznak *Status* je jediným zbývajícím nečíselným příznakem, je typu `object` a nabývá pouze dvou hodnot - *Developing*, *Developed*. Převedu ho na typ `category` a poté na číselnou reprezentaci (indikátor nabývající 0/1) pomocí `cat.codes`.

In [5]:
display(Xtest["Status"].unique())
print()

Xtrain["Status"] = Xtrain["Status"].astype("category")
categories_status = Xtrain["Status"].dtype
Xval["Status"] = Xval["Status"].astype(categories_status)
Xtest["Status"] = Xtest["Status"].astype(categories_status)

Xtrain["Status"] = Xtrain["Status"].cat.codes
Xval["Status"] = Xval["Status"].cat.codes
Xtest["Status"] = Xtest["Status"].cat.codes

Xtrain.info()

array(['Developing', 'Developed'], dtype=object)


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1630 entries, ('Kuwait', np.int64(2011)) to ('Panama', np.int64(2007))
Data columns (total 19 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Status                           1630 non-null   int8   
 1   Adult Mortality                  1630 non-null   float64
 2   infant deaths                    1630 non-null   int64  
 3   Alcohol                          1530 non-null   float64
 4   percentage expenditure           1630 non-null   float64
 5   Hepatitis B                      1313 non-null   float64
 6   Measles                          1630 non-null   int64  
 7   BMI                              1612 non-null   float64
 8   under-five deaths                1630 non-null   int64  
 9   Polio                            1618 non-null   float64
 10  Total expenditure                1512 non-null   float64
 11  Diphtheria                       1

## Chybějící hodnoty

Nyní už jsou všechny příznaky číselné. V mnoha sloupcích jsou chybějící hodnoty. Pro všechny tři množiny je nahradím mediánem z trénovací množiny. Zvolil jsem právě medián, protože na rozdíl od průměru do sloupce *Status* doplní nulu nebo jedničku a význam příznaku zůstane zachován (musím si dávat pozor, jestli není v dataframu stejný počet nul a jedniček, to se tady neděje).

In [6]:
fill_values = Xtrain.median()

Xtrain = Xtrain.fillna(fill_values)
Xval = Xval.fillna(fill_values)
Xtest = Xtest.fillna(fill_values)
print(f"Status doplněn hodnotou: {fill_values['Status']}")

Status doplněn hodnotou: 1.0


## Normalizace

Připravím si normalizovaná data, jednak s využitím min-max normalizace, jednak s využitím standardizace bez transformace indikátoru *Status*.

In [7]:
# min-max normalizace
scaler_minmax = MinMaxScaler()
Xtrain_minmax = Xtrain.copy()
Xval_minmax = Xval.copy()
Xtest_minmax = Xtest.copy()
Xtrain_minmax = scaler_minmax.fit_transform(Xtrain_minmax)
Xval_minmax = scaler_minmax.transform(Xval_minmax)
Xtest_minmax = scaler_minmax.transform(Xtest_minmax)

# standardizace beze zmeny indikatoru Status
scaler_standard = StandardScaler()
cols = Xtrain.columns.difference(["Status"], sort=False)
Xtrain_standard = Xtrain.copy()
Xval_standard = Xval.copy()
Xtest_standard = Xtest.copy()
Xtrain_standard[cols] = scaler_standard.fit_transform(Xtrain_standard[cols])
Xval_standard[cols] = scaler_standard.transform(Xval_standard[cols])
Xtest_standard[cols] = scaler_standard.transform(Xtest_standard[cols])

# Náhodný les

Náhodný les zvládá nelineární vztahy mezi vysvětlujícími a vysvětlovanou proměnnou. Je robustní vůči přeučení díky baggingu, ale je výpočetně náročný.

In [8]:
class CustomRandomForest:
    def __init__(self, n_estimators, max_samples, max_depth, **kwargs):
        """
        n_estimators - počet podmodelů - rozhodovacích stromů.
        max_samples - relativní počet datových bodů, které budou pro každý strom vybrány při trénování (bootstrap)
        max_depth - maximální hloubka každého stromu
        """
        self._n_estimators = n_estimators
        self._max_samples = max_samples
        self._max_depth = max_depth
        self._max_features = kwargs.get("max_features", 1.0)
        self._random_state = kwargs.get("random_state", 1)
        if not isinstance(self._n_estimators, int):
            raise TypeError("n_estimators has to be integer greater than 0")
        if self._n_estimators <= 0:
            raise ValueError("n_estimators has to be integer greater than 0")
        if not isinstance(self._max_samples, float):
            raise TypeError("max_samples has to be float between 0 and 1")
        if not (0 < self._max_samples <= 1):
            raise ValueError("max_samples has to be float between 0 and 1")
        if not isinstance(self._max_depth, int):
            raise TypeError("max_depth has to be integer greater than 0")
        if self._max_depth <= 0:
            raise ValueError("max_depth has to be integer greater than 0")
        if not isinstance(self._max_features, float):
            raise TypeError("max_features has to be float between 0 and 1")
        if not (0 < self._max_features <= 1):
            raise ValueError("max_features has to be float between 0 and 1")
        if not isinstance(self._random_state, int):
            raise TypeError("random_state has to be integer")
        self._trees = []
        
    def fit(self, X, y):
        np.random.seed(self._random_state)
        num_rows = X.shape[0]
        max_samples = math.ceil(num_rows * self._max_samples)
        for i in range(self._n_estimators):
            indices = np.random.choice(num_rows, max_samples, replace=True)
            X_samples = X.iloc[indices] if isinstance(X, (pd.DataFrame, pd.Series)) else X[indices]
            Y_samples = y.iloc[indices] if isinstance(y, (pd.DataFrame, pd.Series)) else y[indices]
            clfDT = DecisionTreeRegressor(max_depth=self._max_depth, max_features=self._max_features, random_state=(i+1)*self._random_state)
            clfDT.fit(X_samples, Y_samples)
            self._trees.append(clfDT)
        
    def predict(self, X):
        ypredicted = np.zeros((X.shape[0], self._n_estimators))
        for i, tree in enumerate(self._trees):
            ypredicted[:, i] = tree.predict(X)
        return ypredicted.mean(axis=1)


Budu ladit následující hyperparametry:

- max_depth - maximální hloubka stromů
- n_estimators - počet stromů
- max_samples - relativní počet bodů, které budou při trénování každého stromu náhodně vybrány
- max_features - relativní počet příznaků, na které se každý strom omezí

In [9]:
param_gridRF = {
    "max_depth": range(1, 22),
    "n_estimators": range(20, 60, 5),
    "max_samples": [1.0, 0.75, 0.5, 0.25],
    "max_features": [1.0, 0.75, 0.5, 0.25],
}
param_combRF = ParameterGrid(param_gridRF)

val_rmse = []
for params in tqdm(param_combRF):
    clfRF = CustomRandomForest(random_state=random_seed, **params)
    clfRF.fit(Xtrain, Ytrain)
    val_rmse.append(metrics.root_mean_squared_error(Yval, clfRF.predict(Xval)))
best_paramsRF = param_combRF[np.argmin(val_rmse)]
display(best_paramsRF)

100%|█████████████████████████████████████████████████████████████████████| 2688/2688 [13:14<00:00,  3.38it/s]


{'n_estimators': 50, 'max_samples': 1.0, 'max_features': 0.5, 'max_depth': 19}

In [10]:
clfRF = CustomRandomForest(random_state=random_seed, **best_paramsRF)
clfRF.fit(Xtrain, Ytrain)
print("Náhodný les")
print(f"Train RMSE: {metrics.root_mean_squared_error(Ytrain, clfRF.predict(Xtrain)):.4f}")
print(f"Validation RMSE: {metrics.root_mean_squared_error(Yval, clfRF.predict(Xval)):.4f}")
print(f"Validation MAE: {metrics.mean_absolute_error(Yval, clfRF.predict(Xval)):.4f}")

Náhodný les
Train RMSE: 0.7412
Validation RMSE: 1.9260
Validation MAE: 1.2617


Náhodnému lesu vyšlo s nejlepšími hodnotami hyperparametrů na validační množině RMSE 1.926 a MAE 1.262. Hodnoty hyperparametrů jsou:

- n_estimators: 50
- max_samples: 1.0
- max_features: 0.5
- max_depth: 19

# Lineární regrese

Lineární regrese předpokládá lineární vztah mezi vysvětlujícími a vysvětlovanou proměnnou, což může být omezující. Na druhou stranu je rychlá, snadno interpretovatelná, ale citlivá na odlehlé hodnoty. Nemá žádné hyperparametry k ladění a nevyžaduje normalizaci dat.

In [11]:
clfLinReg = LinearRegression()
clfLinReg.fit(Xtrain, Ytrain)

print("Lineární regresse")
print(f"Train RMSE: {metrics.root_mean_squared_error(Ytrain, clfLinReg.predict(Xtrain)):.4f}")
print(f"Validation RMSE: {metrics.root_mean_squared_error(Yval, clfLinReg.predict(Xval)):.4f}")
print(f"Validation MAE: {metrics.mean_absolute_error(Yval, clfLinReg.predict(Xval)):.4f}")

Lineární regresse
Train RMSE: 4.0313
Validation RMSE: 3.9167
Validation MAE: 2.9972


Pro lineární regresi vychází na validační množině RMSE 3.917 a validační MAE 2.997

# Metoda nejbližších sousedů (KNN)

KNN je vhodné pro menší datasety, protože výpočetní náročnost rychle roste s velikostí datasetu. Bez normalizace dat by příznaky s velkými hodnotami převážily ty s nízkými, na které by potom byl brán malý ohled. Proto provedu normalizaci dat, vyzkouším standardizaci i min-max normalizaci, mezi nimi vyberu lepší model podle RMSE na validační množině. Budu ladit následující hyperparametry:

- n_neighbors - počet nejbližších sousedů, které brát v potaz
- weights - zda mají bližší sousedi větší váhu, nebo ne
- p - eukleidovská, nebo manhattanská vzdálenost

## Standardizace

In [12]:
param_gridKNN = {
    "n_neighbors": range(1, 40),
    "weights": ["uniform", "distance"],
    "p": [1, 2],
}
param_combKNN = ParameterGrid(param_gridKNN)

val_rmse = []
for params in param_combKNN:
    clfKNN_std = KNeighborsRegressor(n_jobs=-1, **params)
    clfKNN_std.fit(Xtrain_standard, Ytrain)
    val_rmse.append(metrics.root_mean_squared_error(Yval, clfKNN_std.predict(Xval_standard)))
best_paramsKNN_std = param_combKNN[np.argmin(val_rmse)]
display(best_paramsKNN_std)

{'weights': 'distance', 'p': 1, 'n_neighbors': 4}

In [13]:
clfKNN_std = KNeighborsRegressor(n_jobs=-1, **best_paramsKNN_std)
clfKNN_std.fit(Xtrain_standard, Ytrain)
print("KNN (Standardizace)")
print(f"Train RMSE: {metrics.root_mean_squared_error(Ytrain, clfKNN_std.predict(Xtrain_standard)):.4f}")
print(f"Validation RMSE: {metrics.root_mean_squared_error(Yval, clfKNN_std.predict(Xval_standard)):.4f}")
print(f"Validation MAE: {metrics.mean_absolute_error(Yval, clfKNN_std.predict(Xval_standard)):.4f}")

KNN (Standardizace)
Train RMSE: 0.0000
Validation RMSE: 2.1331
Validation MAE: 1.3843


Nejlepší kombinace hyperparametrů má na validační množině RMSE 2.133 a MAE 1.384. Hodnoty hyperparametrů jsou:

- weights: distance
- p: 1
- n_neighbors: 4

Vyzkouším i min-max normalizaci.

## Min-max normalizace

In [14]:
val_rmse = []
for params in param_combKNN:
    clfKNN_minmax = KNeighborsRegressor(n_jobs=-1, **params)
    clfKNN_minmax.fit(Xtrain_minmax, Ytrain)
    val_rmse.append(metrics.root_mean_squared_error(Yval, clfKNN_minmax.predict(Xval_minmax)))
best_paramsKNN_minmax = param_combKNN[np.argmin(val_rmse)]
display(best_paramsKNN_minmax)

{'weights': 'distance', 'p': 1, 'n_neighbors': 3}

In [15]:
clfKNN_minmax = KNeighborsRegressor(n_jobs=-1, **best_paramsKNN_minmax)
clfKNN_minmax.fit(Xtrain_minmax, Ytrain)
print("KNN (Min-max normalizace)")
print(f"Train RMSE: {metrics.root_mean_squared_error(Ytrain, clfKNN_minmax.predict(Xtrain_minmax)):.4f}")
print(f"Validation RMSE: {metrics.root_mean_squared_error(Yval, clfKNN_minmax.predict(Xval_minmax)):.4f}")
print(f"Validation MAE: {metrics.mean_absolute_error(Yval, clfKNN_minmax.predict(Xval_minmax)):.4f}")

KNN (Min-max normalizace)
Train RMSE: 0.0000
Validation RMSE: 2.3581
Validation MAE: 1.5338


KNN využívající min-max normalizaci má s nejlepšími hodnotami hyperparametrů na validační množině RMSE 2.358 a MAE 1.533. To je o něco horší než se standardizací. Hodnoty hyperparametrů jsou:

- weights: distance
- p: 1
- n_neighbors: 3

Všimněme si, že RMSE na trénovací množině je pro oba druhy normalizace rovno nule. To je způsobené tím, že hyperparametr *weights* má hodnotu *distance*, tedy blízké body mají větší váhu. V trénovací množině je ale nejbližší bod každého bodu ten bod samotný, který je sám od sebe vzdálený 0, takže přebije ostatní body a bude se predikovat právě jeho hodnotou.

# Rozhodovací strom

Rozhodovací strom je schopný interpretovat nelineární vztahy mezi vysvětlujícími a vysvětlovanou proměnnou, nevyžaduje normalizaci dat. Má ale tendenci k přeučení, tomu se pokusím zabránit laděním následujících hyperparametrů:

- max_depth - maximální hloubka stromu
- criterion - zda se kvalita rozdělení dat v listu hodnotí podle MSE, nebo MAE (jelikož model vybírám podle RMSE, dává smysl použít MSE, ale zkusím to i s MAE a uvidíme, jaký criterion vyjde lépe)
- min_samples_split - minimální počet vzorků v listu, aby se ještě rozdělovalo dál
- min_samples_leaf - minimální počet vzorků, které musí být v listu

In [16]:
param_gridDT = {
    "max_depth": range(1, 12),
    "criterion": ["squared_error", "absolute_error"],
    "min_samples_split": range(2, 10),
    "min_samples_leaf": range(1, 10),
}
param_combDT = ParameterGrid(param_gridDT)

val_rmse = []
for params in tqdm(param_combDT):
    clfDT = DecisionTreeRegressor(random_state=random_seed, **params)
    clfDT.fit(Xtrain, Ytrain)
    val_rmse.append(metrics.root_mean_squared_error(Yval, clfDT.predict(Xval)))
best_paramsDT = param_combDT[np.argmin(val_rmse)]
display(best_paramsDT)

100%|█████████████████████████████████████████████████████████████████████| 1584/1584 [02:14<00:00, 11.75it/s]


{'min_samples_split': 2,
 'min_samples_leaf': 7,
 'max_depth': 9,
 'criterion': 'squared_error'}

In [17]:
clfDT = DecisionTreeRegressor(random_state=random_seed, **best_paramsDT)
clfDT.fit(Xtrain, Ytrain)
print("Rozhodovací strom")
print(f"Train RMSE: {metrics.root_mean_squared_error(Ytrain, clfDT.predict(Xtrain)):.4f}")
print(f"Validation RMSE: {metrics.root_mean_squared_error(Yval, clfDT.predict(Xval)):.4f}")
print(f"Validation MAE: {metrics.mean_absolute_error(Yval, clfDT.predict(Xval)):.4f}")

Rozhodovací strom
Train RMSE: 1.7330
Validation RMSE: 2.5983
Validation MAE: 1.7981


Nejlepší kombinace hyperparametrů má na validační množině RMSE 2.598 a MAE 1.798. Hodnoty hyperparametrů jsou:

- min_samples_split: 2
- min_samples_leaf: 7
- max_depth: 9
- criterion: squared_error

# Finální model

Nejmenší RMSE na validační množině bylo 1.926 pro náhodný les s hodnotami hyperparametrů níže, proto to bude můj finální model.

- n_estimators: 50
- max_samples: 1.0
- max_features: 0.5
- max_depth: 19

Na testovací množině, kterou model doposud neviděl, odhadnu RMSE na nových datech. Poté udělám predikci hodnot příznaku *Life expectancy* pro data z `evaluation.csv`.

In [18]:
print(f"Odhad RMSE na nových datech: {metrics.root_mean_squared_error(Ytest, clfRF.predict(Xtest)):.4f}")

Odhad RMSE na nových datech: 2.0855


Načtu si data z `evaluation.csv` a zpracuji je stejně jako trénovací, validační a testovací množinu.

In [19]:
Xeval = pd.read_csv("evaluation.csv")
Xeval = Xeval.set_index(["Country", "Year"], verify_integrity=True)
Xeval["Status"] = Xeval["Status"].astype(categories_status)
Xeval["Status"] = Xeval["Status"].cat.codes
Xeval = Xeval.fillna(fill_values)
Xeval.info()
Xeval

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 210 entries, ('Albania', np.int64(2015)) to ('Zambia', np.int64(2010))
Data columns (total 19 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Status                           210 non-null    int8   
 1   Adult Mortality                  210 non-null    float64
 2   infant deaths                    210 non-null    int64  
 3   Alcohol                          210 non-null    float64
 4   percentage expenditure           210 non-null    float64
 5   Hepatitis B                      210 non-null    float64
 6   Measles                          210 non-null    int64  
 7   BMI                              210 non-null    float64
 8   under-five deaths                210 non-null    int64  
 9   Polio                            210 non-null    float64
 10  Total expenditure                210 non-null    float64
 11  Diphtheria                       21

Status  Adult Mortality  infant deaths  Alcohol  \
Country Year                                                    
Albania 2015       1             74.0              0     4.60   
        2014       1              8.0              0     4.51   
        2013       1             84.0              0     4.76   
        2012       1             86.0              0     5.14   
        2011       1             88.0              0     5.37   
...              ...              ...            ...      ...   
Zambia  2014       1            314.0             28     0.01   
        2013       1            328.0             29     2.41   
        2012       1            349.0             29     2.59   
        2011       1            366.0             29     2.57   
        2010       1            363.0             30     2.47   

              percentage expenditure  Hepatitis B  Measles   BMI  \
Country Year                                                       
Albania 2015              364.975229         99.0        0  58.0   
        2014              428.749067         98.0        0  57.2   
        2013              430.876979         99.0        0  56.5   
        2012              412.443356         99.0        9  55.8   
        2011              437.062100         99.0       28  55.1   
...                              ...          ...      ...   ...   
Zambia  2014              196.667577         86.0        9  22.8   
        2013               20.623063         79.0       35  22.3   
        2012              196.915250         78.0      896  21.7   
        2011              183.046169         81.0    13234  21.2   
        2010              184.364910         83.0    15754   2.7   

              under-five deaths  Polio  Total expenditure  Diphtheria  \
Country Year                                                            
Albania 2015                  0   99.0               6.00        99.0   
        2014                  1   98.0               5.88        98.0   
        2013                  1   99.0               5.66        99.0   
        2012                  1   99.0               5.59        99.0   
        2011                  1   99.0               5.71        99.0   
...                         ...    ...                ...         ...   
Zambia  2014                 41   78.0               4.99        86.0   
        2013                 42   74.0               4.99        79.0   
        2012                 43    7.0               4.91        78.0   
        2011                 44   83.0               4.26        81.0   
        2010                 45    8.0               4.41        83.0   

              HIV/AIDS          GDP  Population  thinness  1-19 years  \
Country Year                                                            
Albania 2015       0.1  3954.227830     28873.0                   1.2   
        2014       0.1  4575.763787    288914.0                   1.2   
        2013       0.1  4414.723140    289592.0                   1.3   
        2012       0.1  4247.614380      2941.0                   1.3   
        2011       0.1  4437.178680    295195.0                   1.4   
...                ...          ...         ...                   ...   
Zambia  2014       4.3  1738.882200   1562974.0                   6.3   
        2013       4.8   185.793359   1515321.0                   6.4   
        2012       5.6  1734.936120  14699937.0                   6.5   
        2011       6.3  1644.619672  14264756.0                   6.6   
        2010       6.8  1463.213573    138533.0                   6.7   

              thinness 5-9 years  Income composition of resources  Schooling  
Country Year                                                                  
Albania 2015                 1.3                            0.762       14.2  
        2014                 1.3                            0.761       14.2  
        2013                 1.4                            0.759  

Provedu predikci a výsledek uložím do `results.csv`.

In [20]:
Ypred = clfRF.predict(Xeval)
results_df = pd.DataFrame({"Life expectancy": Ypred}, index=Xeval.index)
results_df.to_csv("results.csv")
results_df

Life expectancy
Country Year                 
Albania 2015        76.007804
        2014        75.759983
        2013        75.828150
        2012        76.023209
        2011        76.262959
...                       ...
Zambia  2014        60.014000
        2013        59.278000
        2012        57.174000
        2011        56.608000
        2010        55.338000

[210 rows x 1 columns]

# Závěr

Dataset jsem zpracoval, abych mohl pomocí náhodného lesa, lineární regrese, metody nejbližších sousedů a rozhodovacího stromu predikovat věk dožití. Nejlépe dopadl náhodný les, který jsem si sám naimplementoval. Zvolil jsem si ho jako svůj finální model a predikci pro data z `evaluation.csv` provedl pomocí něj.